In [1]:
!mkdir -p data
!wget -nc https://data.binance.vision/data/spot/monthly/klines/XRPUSDT/1s/XRPUSDT-1s-2025-02.zip -O data/XRPUSDT-1s-2025-02.zip

File ‘data/XRPUSDT-1s-2025-02.zip’ already there; not retrieving.


In [ ]:
import zipfile

import numpy as np

import binance_column_names as bcn

with zipfile.ZipFile('data/XRPUSDT-1s-2025-02.zip', 'r') as zf:
	with zf.open('XRPUSDT-1s-2025-02.csv') as f:
		data = np.loadtxt(
			f,
			delimiter=',',
			usecols=range(0, 6), # open time, open, high, low, close, volume
			dtype={
				'names': bcn.names[:6],
				'formats': [np.int64] + [np.float64] * 5,
			},
			# skiprows=5*24*60*60,
			max_rows=17*24*60*60,
		)

In [3]:
!cmake -DCMAKE_EXPORT_COMPILE_COMMANDS:BOOL=TRUE -Bbuild && cmake --build build --target rnn_trader

import ctypes

lib =  ctypes.CDLL('./build/librnn_trader.so')

n_var = ctypes.c_size_t.in_dll(lib, 'rnn_trader_var').value
n_obj = ctypes.c_size_t.in_dll(lib, 'rnn_trader_obj').value
n_stat = ctypes.c_size_t.in_dll(lib, 'rnn_trader_stat').value
rnn_trader_candles = ctypes.c_void_p.in_dll(lib, 'rnn_trader_candles')
rnn_trader_candlle_count = ctypes.c_size_t.in_dll(lib, 'rnn_trader_candlle_count')

rnn_trader_run = lib.rnn_trader_run
rnn_trader_run.restype = None
rnn_trader_run.argtypes = [ctypes.c_double * n_var, ctypes.c_double * n_obj, ctypes.c_double * n_stat]

rnn_trader_candles.value = data.ctypes.data_as(ctypes.c_void_p).value
rnn_trader_candlle_count.value = data.shape[0]

-- Configuring done
-- Generating done
-- Build files have been written to: /home/user/projects/algotrade/build
Consolidate compiler generated dependencies of target rnn_trader
[100%] Built target rnn_trader


In [ ]:
from IPython.display import display_markdown
import pathlib
from multiprocessing.pool import ThreadPool as Pool

pathlib.Path('params').mkdir(exist_ok=True)
pool = Pool()

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.evaluator import Evaluator
from pymoo.core.problem import Problem
from pymoo.core.termination import NoTermination
from pymoo.operators.crossover.ux import UniformCrossover
from pymoo.operators.survival.rank_and_crowding import RankAndCrowding
from pymoo.problems.static import StaticProblem

problem = Problem(
	n_var=n_var,
	n_obj=n_obj,
	n_constr=0,
	xl=-3,
	xu=+3,
)

algorithm = NSGA2(
	pop_size=60,
	crossover=UniformCrossover(prob=1.0),
	survival=RankAndCrowding(crowding_func='mnn'),
)
algorithm.setup(problem, termination=NoTermination());

In [5]:
# rerun for as many generations as you need
pop = algorithm.ask()
if algorithm.n_gen == 1:
	try:
		for p, r in zip(pop, np.loadtxt(f'params/{n_var}.txt', delimiter=',', ndmin=2)): p.X = r
	except FileNotFoundError: pass

X = pop.get('X')
F = np.empty((X.shape[0], n_obj))
stat = np.empty((X.shape[0], n_stat))

pool.starmap(rnn_trader_run, zip(np.ctypeslib.as_ctypes(X), np.ctypeslib.as_ctypes(F), np.ctypeslib.as_ctypes(stat)))
Evaluator().eval(StaticProblem(problem, F=F, stat=stat), pop)
algorithm.tell(infills=pop)

res = algorithm.result()

In [6]:
wps, mdds, n_poss = -res.F[:, 0], res.F[:, 1] * 100, np.rint(np.exp(-res.F[:, 2]) - 1).astype(int)
tot_profits = res.pop.get('stat')[:, 0]

output = f'gen: {algorithm.n_gen - 1}  worst-profitable: {(wps > 0).sum()}\n'
output += '| worst profit | mdd | n_pos | tot. profit |\n| --- | --- | --: | --- |\n'
for worst_profit, mdd, n_pos, tot_profit in zip(wps, mdds, n_poss, tot_profits):
	if not ((worst_profit > -.1 and n_pos >= 3) or tot_profit > 0): continue

	output += f'| {worst_profit:.4f} | {mdd:.4f}% | {n_pos} | {tot_profit:.4f} |\n'

display_markdown(output, raw=True)
# with open(f'params/{n_var}.txt', 'w') as f: np.savetxt(f, res.X[(wps > 0)], delimiter=', ')

gen: 1  worst-profitable: 2
| worst profit | mdd | n_pos | tot. profit |
| --- | --- | --: | --- |
| 0.1493 | 2.4618% | 3 | 2.7003 |
| 0.0801 | 3.9534% | 17 | 6.4030 |
